This notebook shows an example recommendation system using doc2vec. We will use a dataset called CMU Book summaries [dataset](http://www.cs.cmu.edu/~dbamman/booksummaries.html). Alternateively, the dataset's link can be found in the `BookSummaries_Link.md` file under the Data folder in Ch7.


In [ ]:
!pip install gensim
!pip install nltk
#todo: add pip for downloading nltk data?

    100% |████████████████████████████████| 24.2MB 930kB/s ta 0:00:011   41% |█████████████▎                  | 10.1MB 5.2MB/s eta 0:00:03
    100% |████████████████████████████████| 112kB 3.4MB/s ta 0:00:01
    100% |████████████████████████████████| 26.0MB 1.0MB/s ta 0:00:011
    100% |████████████████████████████████| 20.0MB 1.8MB/s eta 0:00:01
    100% |████████████████████████████████| 1.4MB 4.6MB/s eta 0:00:01
    100% |████████████████████████████████| 133kB 6.7MB/s ta 0:00:01
  Using cached https://files.pythonhosted.org/packages/a3/43/1e939e1fcd87b827fe192d0c9fc25b48c5b3368902bfb913de7754b0dc03/jmespath-0.9.5-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/69/79/e6afb3d8b0b4e96cefbdc690f741d7dd24547ff1f94240c997a26fa908d3/s3transfer-0.3.3-py2.py3-none-any.whl


In [2]:
from nltk.tokenize import word_tokenize
from gensim.models.doc2vec import Doc2Vec, TaggedDocument


In [3]:
#Read the dataset’s README to understand the data format. 
data_path = "booksummaries.txt"
mydata = {} #titles-summaries dictionary object
for line in open(data_path, encoding="utf-8"):
    temp = line.split("\t")
    mydata[temp[2]] = temp[6]



In [4]:
#prepare the data for doc2vec, build and save a doc2vec model
train_doc2vec = [TaggedDocument((word_tokenize(mydata[t])), tags=[t]) for t in mydata.keys()]
model = Doc2Vec(vector_size=50, alpha=0.025, min_count=10, dm =1, epochs=100)
model.build_vocab(train_doc2vec)
model.train(train_doc2vec, total_examples=model.corpus_count, epochs=model.epochs)
model.save("d2v.model")



In [5]:
#Use the model to look for similar texts
model= Doc2Vec.load("d2v.model")

#This is a sentence from the summary of “Animal Farm” on Wikipedia:
#https://en.wikipedia.org/wiki/Animal_Farm
sample = """
Napoleon enacts changes to the governance structure of the farm, replacing meetings with a committee of pigs who will run the farm.
 """
new_vector = model.infer_vector(word_tokenize(sample))
sims = model.docvecs.most_similar([new_vector]) #gives 10 most similar titles
print(sims)

[('Animal Farm', 0.6777619123458862), ('The Wild Irish Girl', 0.6119967699050903), ("Snowball's Chance", 0.60667884349823), ('Family Matters', 0.5831906199455261), ('Settlers in Canada', 0.582908570766449), ('Poor White', 0.5771366953849792), ('The Road to Omaha', 0.576944887638092), ('Ponni', 0.5766265988349915), ("Family Guy: Stewie's Guide to World Domination", 0.5674009323120117), ('Texas Fever', 0.5643234848976135)]
